# Exact-Match Queries

Use `query_device()` for exact field matching on brand names, product codes, and regulatory numbers.

## Setup

In [ ]:
from pymaude import MaudeDatabase
import pandas as pd

# Use shared database
db = MaudeDatabase('notebooks.db', verbose=True)
db.add_years('2020-2023', tables=['device'], download=True)

print("Setup complete!")

## Exact vs Boolean Search

**Boolean search** (`search_by_device_names`) finds substrings:

In [ ]:
db.create_search_index()

# Finds anything containing "pacemaker"
partial = db.search_by_device_names('pacemaker')
print(f"Boolean search 'pacemaker': {len(partial)} events")
print(f"\nGeneric names found (sample):")
print(partial['GENERIC_NAME'].value_counts().head())

**Exact query** (`query_device`) matches entire field:

In [ ]:
# Only events where GENERIC_NAME exactly equals "Pacemaker"
exact = db.query_device(generic_name='Pacemaker')
print(f"Exact query (generic_name='Pacemaker'): {len(exact)} events")
print(f"\nUnique generic names:")
print(exact['GENERIC_NAME'].unique())

## Query by Brand Name

Case-insensitive exact match:

In [ ]:
# Query specific insulin pump brand
results = db.query_device(brand_name='670G INSULIN PUMP MMT-1780KL')

print(f"Events: {len(results)}")
print(f"\nManufacturer:")
print(results['MANUFACTURER_D_NAME'].value_counts())
print(f"\nGeneric name:")
print(results['GENERIC_NAME'].value_counts())

## Query by Generic Name

Find all brands of a device type:

In [ ]:
# All pacemakers
pacemakers = db.query_device(generic_name='Pacemaker')
print(f"Total pacemaker events: {len(pacemakers)}")
print(f"\nTop brands:")
print(pacemakers['BRAND_NAME'].value_counts().head(10))

## Query by Manufacturer

In [ ]:
medtronic = db.query_device(manufacturer_name='Medtronic')
print(f"Medtronic events: {len(medtronic)}")
print(f"\nTop device types:")
print(medtronic['GENERIC_NAME'].value_counts().head(5))

## Query by Product Code

Product codes are FDA's device classification:

In [ ]:
# DQY = Short-term intravascular catheters
catheters = db.query_device(product_code='DQY')
print(f"Product code DQY events: {len(catheters)}")
print(f"\nTop generic names:")
print(catheters['GENERIC_NAME'].value_counts().head(5))

## Combining Parameters (AND Logic)

Multiple parameters are combined with AND:

In [ ]:
# Specific brand from specific manufacturer
specific = db.query_device(
    generic_name='Pacemaker',
    manufacturer_name='Medtronic',
    start_date='2022-01-01'
)

print(f"Medtronic pacemakers (2022+): {len(specific)} events")
if len(specific) > 0:
    print(f"\nBrands:")
    print(specific['BRAND_NAME'].value_counts().head())

## When to Use Each Method

**Use `query_device()` (exact) when:**
- You know the exact brand/generic/manufacturer name
- You need product code or PMA/PMN queries
- You want precise, unambiguous results

**Use `search_by_device_names()` (boolean) when:**
- You're exploring and don't know exact names
- You want all variations (e.g., all devices containing "pump")
- You need complex boolean logic
- You want to compare multiple device categories

**Note**: MAUDE entries are inconsistent - brand_name, generic_name, and manufacturer_d_name may contain any combination of brand, model, type, or manufacturer. Boolean search is often more reliable.

## Example: Explore First, Then Query

Use boolean search to discover exact field values, then use exact queries:

In [ ]:
# Step 1: Explore
explore = db.search_by_device_names('insulin pump')
print(f"Found {len(explore)} events\n")

print("Top generic names:")
print(explore['GENERIC_NAME'].value_counts().head(3))

# Step 2: Use exact query with discovered value
top_generic = explore['GENERIC_NAME'].value_counts().index[0]
exact_results = db.query_device(generic_name=top_generic)

print(f"\nExact query for '{top_generic}': {len(exact_results)} events")

## Cleanup

In [ ]:
db.close()

## Summary

- `query_device()` uses **exact, case-insensitive matching**
- Query by: `brand_name`, `generic_name`, `manufacturer_name`, `product_code`, `pma_pmn`
- Multiple parameters = **AND logic**
- Use for **precision** when you know exact values
- **Explore with boolean search first**, then refine with exact queries

**Next**: [04_analysis_helpers.ipynb](04_analysis_helpers.ipynb) - Statistical analysis and visualization